# CIELab Sampler

## Parameters

Insert the number of samples of CIELab color space you want and an image with all the colors will be produced. When a color is not present in the RGB color space, then an approximation will be given. Change the variable `n_samples`:

In [12]:
n_samples = 6 ** 4  # Change this value to generate a different number of samples

n_samples

1296

Run the following cell for the sampling and image generation

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage import color
import warnings
from math import ceil, log2
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

min_hex_digits = ceil(log2(max(n_samples, 1)) / 4)


def generate_uniform_lab_samples(n_samples):
    """
    Generate uniformly distributed samples in the CIELab color space and their corresponding range extremes.
    """
    # Calculate the number of points per dimension based on the cubic root of the total number of desired samples
    points_per_dim = int(round(n_samples ** (1 / 3)))

    # Define the ranges for L, a, b in CIELab
    L_range = (0, 100)
    a_range = (-128, 127)
    b_range = (-128, 127)

    L_vals = np.linspace(*L_range, points_per_dim)
    a_vals = np.linspace(*a_range, points_per_dim)
    b_vals = np.linspace(*b_range, points_per_dim)

    # Create a grid of points in the CIELab space
    lab_samples = np.array(np.meshgrid(L_vals, a_vals, b_vals)).T.reshape(-1, 3)

    # Determine the ranges for each dimension based on points_per_dim
    L_step = (L_range[1] - L_range[0]) / (points_per_dim - 1) if points_per_dim > 1 else 0
    a_step = (a_range[1] - a_range[0]) / (points_per_dim - 1) if points_per_dim > 1 else 0
    b_step = (b_range[1] - b_range[0]) / (points_per_dim - 1) if points_per_dim > 1 else 0

    # Calculate range extremes for each sampled point
    extremes = [((L-L_step/2, L+L_step/2), (a-a_step/2, a+a_step/2), (b-b_step/2, b+b_step/2)) for L, a, b in lab_samples]

    # If we have more points than we want, randomly sample from the generated points
    if len(lab_samples) > n_samples:
        selected_indices = np.random.choice(len(lab_samples), size=n_samples, replace=False)
        lab_samples = lab_samples[selected_indices]
        extremes = [extremes[i] for i in selected_indices]

    return lab_samples, extremes


def lab_to_rgb(lab_arr):
    """Convert a batch of CIELab values to RGB and mark approximations."""
    rgb_arr_255 = np.zeros((lab_arr.shape[0], 3), dtype=int)
    approx_flags = np.zeros(lab_arr.shape[0], dtype=bool)

    # Convert each CIELab color to RGB individually and catch warnings
    for i, lab in enumerate(lab_arr):
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")

            # Convert CIELab to RGB
            rgb = color.lab2rgb(lab.reshape(1, 1, 3)).reshape(3)
            rgb_255 = (255 * np.clip(rgb, 0, 1)).astype(int)
            rgb_arr_255[i, :] = rgb_255

            # Mark the color as approximated if relevant warnings are raised
            if any(issubclass(warn.category, UserWarning) and "negative Z values" in str(warn.message) for warn in w):
                approx_flags[i] = True
                print(str(rgb_255) + " approx")

    return rgb_arr_255, approx_flags


def rgb_to_lab(rgb):
    """Convert an RGB color to CIELab color space."""
    # Normalize RGB values to 0-1 range if they are in 0-255 range
    normalized_rgb = [value / 255.0 for value in rgb]
    srgb = sRGBColor(*normalized_rgb, is_upscaled=False)
    lab = convert_color(srgb, LabColor)

    return lab


def hex_to_rgb(hex):
    """
    Convert a Hex color code to an RGB tuple.
    """
    # Strip the "#" prefix if it exists
    hex_color = hex.lstrip('#')
    # Convert hex to RGB values
    rgb = tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))
    return rgb


def rgb_to_hex(rgb):
    """
    Convert an RGB tuple to a Hex color code.
    """
    # Convert each RGB value to its hexadecimal representation and concatenate
    return '#' + ''.join(f'{component:02X}' for component in rgb)


def hex_to_lab(hex):
    """
    Convert a Hex color code to CIELab color space.
    """
    # First, convert Hex to RGB
    rgb = hex_to_rgb(hex)
    # Then, use the rgb_to_lab function to convert RGB to CIELab
    lab = rgb_to_lab(rgb)
    return lab


def create_color_charts_with_spacing(lab_samples, lab_samples_extremes, rgb_samples, approx_flags, 
                                     max_num_colors_per_image=4096, square_size_cm=3, hor_spacing_cm=1, 
                                     label_font_size=10, ver_spacing_cm=1.5, 
                                     save_path_prefix='color_chart'):
    n_samples = len(lab_samples)
    num_images = ceil(n_samples / max_num_colors_per_image)
    
    for img_index in range(num_images):
        start_index = img_index * max_num_colors_per_image
        end_index = min(start_index + max_num_colors_per_image, n_samples)
        current_lab_samples = lab_samples[start_index:end_index]
        current_rgb_samples = rgb_samples[start_index:end_index]
        current_approx_flags = approx_flags[start_index:end_index]
        
        fig_dpi = 100
        vertical_offset_cm = (label_font_size / 3) / fig_dpi * 2.54
        
        num_samples_hor_side = ceil(len(current_lab_samples) ** 0.5)
        num_samples_ver_side = ceil(len(current_lab_samples) / num_samples_hor_side)
        total_hor_size_cm = num_samples_hor_side * (square_size_cm + hor_spacing_cm) + hor_spacing_cm
        total_ver_size_cm = num_samples_ver_side * (square_size_cm + ver_spacing_cm) + ver_spacing_cm       
        fig, ax = plt.subplots(figsize=(total_hor_size_cm, total_ver_size_cm), dpi=fig_dpi)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

        for i, (((lab, rgb), approx), lab_extremes) in enumerate(zip(zip(zip(current_lab_samples, current_rgb_samples), current_approx_flags), lab_samples_extremes)):
            row, col = divmod(i, num_samples_hor_side)
            x_pos = col * (square_size_cm + hor_spacing_cm) + hor_spacing_cm
            y_pos = row * (square_size_cm + ver_spacing_cm) + ver_spacing_cm
            rect = Rectangle((x_pos, y_pos), square_size_cm, square_size_cm, color=rgb / 255)
            ax.add_patch(rect)
            
            compact_lab = ', '.join(f"{x:.6f}".rstrip('0').rstrip('.') if '.' in f"{x:.6f}" else f"{x}" for x in lab)
            compact_rgb = ', '.join(f"{x:.6f}".rstrip('0').rstrip('.') if '.' in f"{x:.6f}" else f"{x}" for x in rgb)
            compact_lab_extremes_1 = ', '.join(f"{x:.6f}".rstrip('0').rstrip('.') if '.' in f"{x:.6f}" else f"{x}" for x in lab_extremes[0])
            compact_lab_extremes_2 = ', '.join(f"{x:.6f}".rstrip('0').rstrip('.') if '.' in f"{x:.6f}" else f"{x}" for x in lab_extremes[1])
            compact_lab_extremes_3 = ', '.join(f"{x:.6f}".rstrip('0').rstrip('.') if '.' in f"{x:.6f}" else f"{x}" for x in lab_extremes[2])
            text_label = f"{i:0{min_hex_digits}X}\n{compact_lab}\n{compact_rgb} — {rgb_to_hex(rgb)}"
            if approx:
                text_label += " approx"
            text_label += f"\n{compact_lab_extremes_1}\n{compact_lab_extremes_2}\n{compact_lab_extremes_3}"

            ax.text(x_pos + square_size_cm / 2, y_pos + square_size_cm + vertical_offset_cm, text_label, 
                    color='black', ha='center', va='top', fontsize=label_font_size, fontweight='bold')

        ax.set_xlim(0, total_hor_size_cm)
        ax.set_ylim(0, total_ver_size_cm)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')

        plt.gca().invert_yaxis()

        save_path = f"{save_path_prefix}_{img_index + 1}.png"
        plt.savefig(save_path, bbox_inches='tight')
        plt.close(fig)


# Samples generation and image generation
lab_samples, lab_samples_extremes  = generate_uniform_lab_samples(n_samples)

rgb_samples, approx_flags = lab_to_rgb(lab_samples)

max_num_colors_per_image = 16 ** 3

create_color_charts_with_spacing(lab_samples, lab_samples_extremes, rgb_samples, approx_flags,
                                 square_size_cm=3, hor_spacing_cm=1, label_font_size=12, 
                                 max_num_colors_per_image=max_num_colors_per_image,
                                 ver_spacing_cm=1.5, save_path_prefix=str(n_samples)+'_sampled_colors')

[147   8   0] approx
[173   0   0] approx
[190   0   0] approx
[190   0   0] approx
[ 0 68  0] approx
[64  0  0] approx
[255   0   0] approx
[  0 120   0] approx
[141   0   0] approx
[ 0 56  0] approx
[ 0 35  0] approx
[207   0   0] approx
[ 0 72  0] approx
[132   0   0] approx
[ 0 72  0] approx
[211  73   0] approx
[255   0   0] approx
[ 0 93  0] approx
[ 0 27  0] approx
[90 68  0] approx
[207   0   0] approx
[ 0 95  0] approx
[ 0 16  0] approx
[ 0 47  0] approx
[45 25  0] approx
[101   0   0] approx
[ 0 35  0] approx
[ 0 46  0] approx
[146 116   0] approx
[34 55  0] approx
[ 0 52  0] approx
[117   0   0] approx
[45 25  0] approx
[ 0 42  0] approx
[122   0   0] approx
[ 0 85  0] approx
[17 34  0] approx
[82  0  0] approx
[ 0 46  0] approx
[178  45   0] approx
[165   0   0] approx
[90 68  0] approx
[ 0 16  0] approx
[88  0  0] approx
[122   0   0] approx
[66  4  0] approx
[234   0   0] approx
[  0 116   0] approx
[47  0  0] approx
[ 0 52  0] approx
[ 0 42  0] approx
[117   0   0] appro

# Nearest sample

Get the nearest sample from a new color in CIELab. Change value in variable `target_lab`:

In [3]:
target_lab = [40., 10., 30.]  # Target CIELab color. Make sure the values are float or it returns black

Run the following cell to retrieve the nearest sample

In [4]:

def find_nearest_color_lab(target_lab, generated_lab_samples):
    """
    Find the nearest CIELab color to the target_lab color among the generated samples using Delta E.
    """
    # Check if target_lab is a LabColor instance, if not, create one
    if not isinstance(target_lab, LabColor):
        target_lab_color = LabColor(*target_lab)
    else:
        target_lab_color = target_lab

    min_distance = float('inf')
    nearest_color = None

    for lab in generated_lab_samples:
        sample_lab_color = LabColor(*lab)
        delta_e = delta_e_cie2000(target_lab_color, sample_lab_color)

        if delta_e < min_distance:
            min_distance = delta_e
            nearest_color = lab

    return nearest_color


def save_comparison_chart(target_lab, nearest_lab, square_size=4, spacing=4, save_path='colors_comparison.png'):
    """
    Save an image with two squares: one for the target color and one for the nearest CIELab color.
    """
    # Extract LAB values if target_lab is a LabColor instance
    if isinstance(target_lab, LabColor):
        target_lab_values = [target_lab.lab_l, target_lab.lab_a, target_lab.lab_b]
    else:
        target_lab_values = target_lab

    if isinstance(nearest_lab, LabColor):
        nearest_lab_values = [nearest_lab.lab_l, nearest_lab.lab_a, nearest_lab.lab_b]
    else:
        nearest_lab_values = nearest_lab

    # Calculate the dimensions of the figure
    fig_width = 2 * square_size + spacing
    fig_height = square_size
    fig, ax = plt.subplots(1, 2, figsize=(fig_width, fig_height), dpi=100)

    # Adjust the position of the subplots
    plt.subplots_adjust(wspace=(spacing / fig_width) * (fig_width / fig_height))

    # Prepare the LAB colors for conversion
    target_lab_array = np.array(target_lab_values).reshape(1, 1, 3)
    nearest_lab_array = np.array(nearest_lab_values).reshape(1, 1, 3)

    # Convert the LAB colors to RGB for display
    target_rgb = color.lab2rgb(target_lab_array).reshape(3, )
    nearest_rgb = color.lab2rgb(nearest_lab_array).reshape(3, )

    # Draw the square for the target color
    rect_target = Rectangle((0, 0), square_size, square_size, color=target_rgb)
    ax[0].add_patch(rect_target)
    ax[0].set_title("Starting Color:\n" + ", ".join([f"{val:.2f}" for val in target_lab_values]))
    ax[0].axis('off')

    # Draw the square for the nearest CIELab color
    rect_nearest = Rectangle((0, 0), square_size, square_size, color=nearest_rgb)
    ax[1].add_patch(rect_nearest)
    ax[1].set_title("Nearest Sample:\n" + ", ".join([f"{val:.2f}" for val in nearest_lab_values]))
    ax[1].axis('off')

    # Save the image
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)


# Find the nearest CIELab color for a given CIELab and RGB color
nearest_lab = find_nearest_color_lab(target_lab, lab_samples)

print("Nearest CIELab color of ", str(target_lab), " is: ", nearest_lab)

save_comparison_chart(target_lab, nearest_lab, save_path='lab_approx.png')

Nearest CIELab color of  [40.0, 10.0, 30.0]  is:  [41.66666667 10.125      31.375     ]


Get the nearest sample from a new color in RGB. Change value in variable `target_rgb`:

In [5]:
target_rgb = [120., 65., 200.]  # Target RGB color

Run the following cell to retrieve the nearest sample

In [6]:
# Find the nearest CIELab color for a given CIELab and RGB color
target_lab_from_rgb = rgb_to_lab(target_rgb)

nearest_lab_from_rgb = find_nearest_color_lab(target_lab_from_rgb, lab_samples)

print("Nearest CIELab color of RGB ", str(target_rgb), " is: ", nearest_lab_from_rgb)

save_comparison_chart(target_lab_from_rgb, nearest_lab_from_rgb, save_path='rgb_approx.png')

Nearest CIELab color of RGB  [120.0, 65.0, 200.0]  is:  [ 41.66666667  52.625      -64.25      ]
